In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [68]:
X_train_raw = mnist.train.images
y_train = mnist.train.labels
X_valid_raw = mnist.validation.images
y_valid = mnist.validation.labels
X_test_raw = mnist.test.images
y_test = mnist.test.labels

In [87]:
X_train = X_train_raw.reshape([X_train.shape[0], 28, 28, 1])
X_valid = X_valid_raw.reshape([X_valid.shape[0], 28, 28, 1])
X_test = X_test_raw.reshape([X_test.shape[0], 28, 28, 1])

In [91]:
n_samples, input_height, input_width, input_channels = X_train.shape
n_outputs = 10

In [127]:
tf.reset_default_graph()

In [128]:
X = tf.placeholder(shape=[None, input_height, input_width, input_channels],
                   dtype=tf.float32,
                   name='X')
y = tf.placeholder(shape=None, dtype=tf.int64, name='X')

In [129]:
conv1 = tf.layers.conv2d(X,
                        filters=32,
                        kernel_size=5,
                        strides=[1,1],
                        padding="SAME",
                        name='conv1',
                        activation=tf.nn.relu)
pool1 = tf.nn.avg_pool(conv1,
                       ksize=[1,2,2,1],
                       strides=[1,2,2,1],
                       padding="VALID",
                       name='pool1')
conv2 = tf.layers.conv2d(X,
                        filters=64,
                        kernel_size=5,
                        strides=[2,2],
                        padding="SAME",
                        name='conv2',
                        activation=tf.nn.relu)
pool2 = tf.nn.avg_pool(conv2,
                       ksize=[1,2,2,1],
                       strides=[1,2,2,1],
                       padding="VALID",
                       name='pool2')
pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
dense1 = tf.layers.dense(pool2_flat, 100, activation=tf.nn.relu, name='dense1')
dropout = tf.layers.dropout(inputs=dense1, rate=0.5)
logits = tf.layers.dense(dropout, n_outputs, name='logits')

In [130]:
# define individual cross entropy and overall loss for a batch
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name='loss')

In [131]:
learning_rate = 0.01
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,
                                   beta1=0.9,
                                   beta2=0.999)
training_step = optimizer.minimize(loss, name='training_step')
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')

In [132]:
n_epochs = 40
batch_size = 50

In [133]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        rand_idx = np.random.permutation(np.arange(n_samples))
        for iteration in range(n_samples // batch_size):
            batch_idx = rand_idx[(iteration*batch_size):((iteration+1)*batch_size)]
            X_batch = X_train[batch_idx,:]
            y_batch = y_train[batch_idx]
            sess.run(training_step, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid,
                                           y: y_valid})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val)

(0, 'Train accuracy:', 0.98000002, 'Val accuracy:', 0.98220003)
(1, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98320001)
(2, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98180002)
(3, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98320001)
(4, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98439997)
(5, 'Train accuracy:', 0.98000002, 'Val accuracy:', 0.98119998)
(6, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98220003)
(7, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98500001)
(8, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98140001)
(9, 'Train accuracy:', 0.98000002, 'Val accuracy:', 0.98460001)
(10, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98720002)
(11, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98379999)
(12, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98379999)
(13, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98619998)
(14, 'Train accuracy:', 0.98000002, 'Val accuracy:', 0.98360002)
(15, 'Train accuracy:', 1.0, 'Val accuracy:', 0.98400003)
(16, 'Train accuracy:', 1.0, 'Val accuracy:', 0.986999